In [8]:
import pandas as pd
from docx import Document


In [9]:
network_df = pd.read_excel("reseau_en_arbre.xlsx")


broken_network_df = network_df[network_df["infra_type"] == "a_remplacer"]


set_id_batiments = set(network_df["id_batiment"].values)
set_id_broken_batiments = set(broken_network_df["id_batiment"].values)

list_id_batiment = []
state_batiment = []

for id_batiment in set_id_batiments:
    list_id_batiment.append(id_batiment)
    if id_batiment in set_id_broken_batiments:
        state_batiment.append("à_reparer")  # bâtiment à réparer
    else:
        state_batiment.append("intact")     # bâtiment intact


state_df = pd.DataFrame({
    "id_bat": list_id_batiment,
    "state_batiment": state_batiment
})

header_row = pd.DataFrame([state_df.columns], columns=state_df.columns)

df_final = pd.concat([header_row, state_df], ignore_index=True)

df_final.to_excel("etat_batiment.xlsx", index=False, header=False)
print("Fichier 'etat_batiment.xlsx' généré avec succès.")


Fichier 'etat_batiment.xlsx' généré avec succès.


1- Il faut recupérer pour un seul infra les differents batiment qu'il permet de raccorder
2- la metrique d'évaluation de la priorité pour le moment est longueur_d'infra/nbr_maison(nbr_prise) métrique de difficulté pour reconstruire un seul infra.
3- pour chaque batiment on a une liste des infras du départ jusqu'au arrivée (de l'origine de l'infra jusqu'au dernier batiment), c'est déjà ordonnée dans le fichier xlsx
4- résultat : 4 phases de reconstruction (4 degrées de priorité pour chaque batiment)
5- il faut résonner en fonction d'un chemin d'infra (et pas par 1 infra parce que ça sert à rien de reconstruire une infra dans la 1ere phase par exemple sans reconstituer l'electricité dans le batiment). du coup la métrique devient : somme(longueur_d'infra/nbr_maison_dependant_de_cette_infra) pour chaque batiment. Exemple :
id_batiment	nb_maisons	infra_id	infra_type	longueur
E000001	4	P007111	infra_intacte	12.3144613356693
E000001	4	P007983	infra_intacte	40.3209288665353
E000001	4	P000308	infra_intacte	39.1407985664577
E000001	4	P007819	infra_intacte	17.3904643240185
6- on va calculer la difficulté de chaque infra , pour faire la somme facilement par batiment.
7- quand on répare un infra il faut mettre à jour la métrique.
8- metrique final cout*duree/nbr_prise par batiment = somme de cout*duree/nbr_prise par infra
9- on va tout simplement dans le tableau excel ajouter une colonne cout en euros et une colonne durée en temps et une colonne type_reparation(aerien/semi-aérien/fourreau)
10- pour la mise a jour c'est simple , a chaque fois qu'une infra est remplacé on la fait sortir de la liste des broken_infra et on recalcule la somme de cout*duree/nbr_prise par infra .


In [10]:
class Infra:

    def __init__(self,infra_id,length,infra_type,nb_house):
        self.infra_id = infra_id
        self.length = length
        self.infra_type = infra_type
        self.nb_house = nb_house

    def __repr__(self):
        return f"Infrastructure {self.infra_id} | Type: {self.infra_type} | Length: {self.length}m | Houses: {self.nb_house}"

    def repair_infra(self):
        pass

    def get_infra_difficulty(self) :
        return self.length/self.nb_house
    

In [11]:
class Building:

    def __init__(self,id_building,list_infra):
        self.id_building = id_building
        self.list_infra = []

    def add_infra(self, infra):
        self.list_infra.append(infra)

    def get_building_difficulty(self) :
        s = 0
        for infra in self.list_infra : s+=infra.get_infra_difficulty()
        return s

    def add_infra(self, infra):
        self.list_infra.append(infra)
        
    def __repr__(self):
        return f"Building {self.id_building} | {self.list_infra}"


In [12]:
#1er script : calcul de la 1ere métrique :

network_df = pd.read_excel("reseau_en_arbre.xlsx")
broken_network_df = network_df[network_df["infra_type"] == "a_remplacer"]


broken_infra = []
buildings_without_electricity = []
infra_difficulties = {}
buildings = {}
building_difficulties = {}

for _,infra in broken_network_df.iterrows():
    infrastructure = Infra(infra_id=infra["infra_id"], length=infra["longueur"], infra_type=infra["infra_type"], nb_house=infra["nb_maisons"])
    broken_infra.append(infrastructure)

print(broken_infra)

for infra, building_id in zip(broken_infra, broken_network_df["id_batiment"]):
    if building_id not in buildings:
        buildings[building_id] = Building(building_id, [])
    buildings[building_id].list_infra.append(infra)

buildings_without_electricity = list(buildings.values())

print(buildings_without_electricity)

for infra in broken_infra :
    infra_difficulties[infra.infra_id] = infra.get_infra_difficulty()

print(infra_difficulties)

for building in buildings_without_electricity :
    building_difficulties[building.id_building] = building.get_building_difficulty()


final_result = dict(sorted(building_difficulties.items(), key=lambda item: item[1]))
print(final_result)




[Infrastructure P007113 | Type: a_remplacer | Length: 12.1935087083631m | Houses: 1, Infrastructure P007984 | Type: a_remplacer | Length: 30.0575979492587m | Houses: 1, Infrastructure P007823 | Type: a_remplacer | Length: 12.290282829583m | Houses: 1, Infrastructure P006193 | Type: a_remplacer | Length: 12.1863879405322m | Houses: 1, Infrastructure P002851 | Type: a_remplacer | Length: 10.8032015604234m | Houses: 1, Infrastructure P007624 | Type: a_remplacer | Length: 4.10990474598487m | Houses: 1, Infrastructure P005500 | Type: a_remplacer | Length: 13.1420349052071m | Houses: 1, Infrastructure P007990 | Type: a_remplacer | Length: 18.7028610321555m | Houses: 1, Infrastructure P007447 | Type: a_remplacer | Length: 2.84535154473363m | Houses: 1, Infrastructure P005531 | Type: a_remplacer | Length: 14.8493095953268m | Houses: 1, Infrastructure P007639 | Type: a_remplacer | Length: 6.13280316452202m | Houses: 1, Infrastructure P005500 | Type: a_remplacer | Length: 13.1420349052071m | Hou

In [13]:
#Redéfinition de la classe Infra
class Infra:

    def __init__(self,infra_id,length,infra_type,nb_house,cost,time):
        self.infra_id = infra_id
        self.length = length
        self.infra_type = infra_type
        self.nb_house = nb_house
        self.cost = cost
        self.time = time

    def __repr__(self):
        return f"Infrastructure {self.infra_id} | Type: {self.infra_type} | Length: {self.length}m | Houses: {self.nb_house} | cost: {self.cost} | time: {self.time}"

    def repair_infra(self) :
        pass

    def get_infra_difficulty(self) :
        return self.cost*self.time/self.nb_house

In [14]:
#2éme script : Calcul de la nouvelle métrique.

#Méthode de lecture et de normalisation du fichier annexe 
def extract_infra_costs_and_times(appendix):
    doc = Document(appendix)
    text = "\n".join([p.text for p in doc.paragraphs])
    
    prix_dict = {}
    duree_dict = {}
    current = None
    
    for line in text.splitlines():
        line = line.strip().lower()
        if not line:
            continue
        if "prix" in line:
            current = "prix"
            continue
        if "durée" in line or "duree" in line:
            current = "duree"
            continue

        if ":" in line:
            type_infra, val = line.split(":", 1)
            type_infra = type_infra.strip()
            val = val.strip().split(" ")[0]
            val_numeric = ''.join(c for c in val if c.isdigit() or c == ".")
            if not val_numeric:  # ignore si rien de numérique
                continue
            if current == "prix":
                prix_dict[type_infra] = float(val_numeric)
            elif current == "duree":
                duree_dict[type_infra] = float(val_numeric)
    
    return prix_dict, duree_dict


#Méthode regroupant tout les informations essentielles au calcul de la métrique :
def complete_data(buildings,infra,network,appendix) :


    buildings_csv = pd.read_csv(buildings)
    infra_csv = pd.read_csv(infra)
    network_xlsx = pd.read_excel(network)
    prix_dict, duree_dict = extract_infra_costs_and_times(appendix)

    
    #Corriger le nombre des maisons pour chaque batiment
    buildings_correction = pd.merge(network_xlsx, buildings_csv, on="id_batiment", how="left", suffixes=("", "_csv1"))
    buildings_correction["nb_maisons"] = buildings_correction["nb_maisons_csv1"].combine_first(buildings_correction["nb_maisons"])
    buildings_correction = buildings_correction.drop(columns=["nb_maisons_csv1"])


    #Ajouter la colonne type_infra pour chaque infra
    infra_type = pd.merge(buildings_correction, infra_csv[["id_infra", "type_infra"]], left_on="infra_id",right_on="id_infra",how="left")
    infra_type = infra_type.drop(columns=["id_infra"])

    #Ajouter les colonnes de coût et durée pour reconstruire chaque infra
    infra_type["cout_total"] = infra_type["type_infra"].str.lower().map(prix_dict) * infra_type["longueur"]
    infra_type["duree_totale"] = infra_type["type_infra"].str.lower().map(duree_dict) * infra_type["longueur"]
    complete_data = infra_type.drop_duplicates()


    return complete_data


#Méthode calculant la métrique de difficulté pour chaque batiment: 
def final_metric(complete_data):

    broken_infra = []
    buildings_without_electricity = []
    buildings = {}
    infra_difficulties = {}
    building_difficulties = {}


    complete_data = complete_data[complete_data["infra_type"] == "a_remplacer"]
    for _,infra in complete_data.iterrows():
        infrastructure = Infra(infra_id=infra["infra_id"], length=infra["longueur"], infra_type=infra["infra_type"], nb_house=infra["nb_maisons"], cost=infra["cout_total"]
                               ,time=infra["duree_totale"])
        broken_infra.append(infrastructure)
    for infra, building_id in zip(broken_infra, complete_data["id_batiment"]):
        if building_id not in buildings:
            buildings[building_id] = Building(id_building=building_id, list_infra=[])
        buildings[building_id].list_infra.append(infra)
    buildings_without_electricity = list(buildings
                                         .values())
    
    for infra in broken_infra :
        infra_difficulties[infra.infra_id] = infra.get_infra_difficulty()
    

    for building in buildings_without_electricity :
        building_difficulties[building.id_building] = building.get_building_difficulty()
    
    final_result = dict(sorted(building_difficulties.items(), key=lambda item: item[1]))
  
    return final_result



In [15]:
#Difficulté de chaque batiment en euros.heure/nbr_maisons
data = complete_data("batiments.csv","infra.csv","reseau_en_arbre.xlsx","annexe.docx")
final_metric(data)


{'E000112': 38379.68795337064,
 'E000364': 44508.741467693544,
 'E000127': 45011.279628416785,
 'E000259': 45759.1342536561,
 'E000098': 55678.96162919077,
 'E000258': 61012.17900487479,
 'E000111': 62290.77769626339,
 'E000274': 63695.351979753395,
 'E000128': 64577.08245465618,
 'E000302': 68142.4824660393,
 'E000222': 71841.09259332197,
 'E000266': 71926.29820878938,
 'E000194': 73214.61480584975,
 'E000225': 73214.61480584975,
 'E000365': 80051.69597182679,
 'E000195': 91518.2685073122,
 'E000160': 93547.70416614751,
 'E000373': 107938.81480704871,
 'E000381': 111463.31918377968,
 'E000268': 119109.43661642613,
 'E000196': 122024.35800974959,
 'E000376': 126111.58203327718,
 'E000378': 136981.5768470732,
 'E000301': 142706.44978791545,
 'E000316': 148004.7665223368,
 'E000293': 150686.87624582407,
 'E000315': 163179.16379889174,
 'E000228': 166486.74171622988,
 'E000377': 170776.25132296403,
 'E000374': 187638.69966126635,
 'E000346': 189172.57231049918,
 'E000322': 198412.46720908

In [16]:
# Constantes
MAX_WORKERS = 4
WORKER_COST_PER_HOUR = 300 / 8  # €/heure
HOSPITAL_MARGIN = 1.2  # marge de sécurité pour l'hôpital (16h de travaux)
GENERATOR_AVAILABLE_HOURS = 20

In [17]:
PHASE_COST_PERCENTAGE = {
    0: 1.0,   # phase 0 = hôpital
    1: 0.4,   # phase 1 = 40% du coût
    2: 0.2,   # phase 2 = 20%
    3: 0.2,   # phase 3 = 20%
    4: 0.2    # phase 4 = 20%
}

In [18]:
#Redéfinition de la classe Infra pour pouvoir calculer la nouvelle métrique
class Infra:
    def __init__(self, infra_id, length, infra_type, nb_house, cost,time):
        self.infra_id = infra_id
        self.length = length
        self.infra_type = infra_type
        self.nb_house = nb_house
        self.cost = cost
        self.time = time

    def get_difficulty(self):
        """Calcul de la difficulté d'une infra"""
        effective_time = self.time / MAX_WORKERS
        difficulty = (effective_time * self.cost) / self.nb_house
        return difficulty

    def __repr__(self):
        return f"Infra({self.infra_id}, type={self.infra_type}, houses={self.nb_house})"

In [32]:
#Redéfinition de la classe Infra pour pouvoir calculer la nouvelle métrique

class Building:
    def __init__(self, id_building):
        self.id_building = id_building
        self.list_infra = []

    def add_infra(self, infra):
        self.list_infra.append(infra)

    def get_building_difficulty(self):
        return sum(i.get_difficulty() for i in self.list_infra if i.infra_type == "a_remplacer")

In [56]:
#Calcul du cout et de la durée essentielle pour la phase 0 et donc la réparation de l'hopital
def phase_0(hopital_infra_list):

    total_effective_time = 0
    total_cost = 0
    total_effective_timee = []

    for infra in hopital_infra_list:
        effective_time = infra.time / MAX_WORKERS
        total_cost += infra.cost
        total_effective_timee.append(effective_time)
        print(f"Infra {infra.infra_id} | Temps effectif : {effective_time:.2f} h | Coût : {infra.cost:.2f} €")
    
    total_effective_time = max(total_effective_timee)


    print(f"\nTemps total estimé : {total_effective_time:.2f} h")
    print(f"Coût total : {total_cost:.2f} €")

    # On vérifie si le générateur tiendra
    if total_effective_time > GENERATOR_AVAILABLE_HOURS:
        print("Attention : le générateur risque de tomber en panne avant la fin des travaux !")
        print(f"Autonomie restante : {GENERATOR_AVAILABLE_HOURS} h | Temps requis : {total_effective_time:.2f} h")
    else:
        print("Le générateur pourra tenir pendant toute la phase 0.")

    return {
        "total_time": total_effective_time,
        "total_cost": total_cost
    }

In [74]:
hospital_rows = data[data["type_batiment"] == "hôpital"]
hospital_rows = hospital_rows[hospital_rows["infra_type"] == "a_remplacer"]
hopital_infra_list = [
    Infra(
        infra_id=row["infra_id"],
        length=row["longueur"],
        infra_type=row["type_infra"],
        nb_house=row["nb_maisons"],
        cost=row["cout_total"],
        time=row["duree_totale"]
    )
    for _, row in hospital_rows.iterrows()
]

result_phase0 = phase_0(hopital_infra_list)



Infra P005500 | Temps effectif : 6.57 h | Coût : 6571.02 €
Infra P007990 | Temps effectif : 9.35 h | Coût : 9351.43 €
Infra P007447 | Temps effectif : 3.56 h | Coût : 2560.82 €

Temps total estimé : 9.35 h
Coût total : 18483.26 €
Le générateur pourra tenir pendant toute la phase 0.


In [45]:
def plan_phases_by_building_once(df):
    # On crée les bâtiments et leur liste d'infra
    buildings = {}
    for _, row in df.iterrows():
        b_id = row["id_batiment"]
        if b_id not in buildings:
            buildings[b_id] = Building(b_id)
        infra = Infra(
            infra_id=row["infra_id"],
            length=row["longueur"],
            infra_type=row["infra_type"],
            nb_house=row["nb_maisons"],
            cost=row["cout_total"],
            time=row["duree_totale"]
        )
        buildings[b_id].add_infra(infra)

    # On calcule la difficulté de chaque bâtiment
    building_difficulties = {b.id_building: b.get_building_difficulty() for b in buildings.values()}

    # On trie les bâtiments par difficulté croissante
    sorted_buildings = sorted(buildings.values(), key=lambda b: b.get_building_difficulty())

    # On initialise les phases
    phases = {1: [], 2: [], 3: [], 4: []}
    PHASE_COST_PERCENTAGE = {1: 0.4, 2: 0.2, 3: 0.2, 4: 0.2}
    phase_index = 1
    assigned_cost = 0
    total_cost = sum(i.cost for b in buildings.values() for i in b.list_infra if i.infra_type == "a_remplacer")

    # On répartit les infra dans les phases en suivant la difficulté des bâtiments
    for building in sorted_buildings:
        building_infra = [i for i in building.list_infra if i.infra_type == "a_remplacer"]
        building_cost = sum(i.cost for i in building_infra)

        # Passer à la phase suivante si le coût cumulé dépasse la limite
        if assigned_cost + building_cost > total_cost * PHASE_COST_PERCENTAGE.get(phase_index, 0):
            phase_index += 1
            if phase_index > 4:
                phase_index = 4
            assigned_cost = 0

        phases[phase_index].extend(building_infra)
        assigned_cost += building_cost

    # Difficulté de chaque infra
    infra_difficulties = {
        i.infra_id: i.get_difficulty()
        for b in buildings.values() for i in b.list_infra
    }

    # On affiche les résultas résumé
    for idx in range(1, 5):
        phase_infra = phases[idx]
        print(f"\n Phase {idx} | Infra à remplacer : {len(phase_infra)}")
        for i in phase_infra:
            print(f"  {i.infra_id} | Difficulté : {i.get_difficulty():.2f} | "
                  f"Coût : {i.cost:.2f} | Durée : {i.time:.2f} h")

    return phases, building_difficulties, infra_difficulties

In [48]:
phases_result,building_difficulties,infra_difficulties  = plan_phases_by_building_once(data)  
print(building_difficulties)
print(infra_difficulties)




 Phase 1 | Infra à remplacer : 229
  P005664 | Difficulté : 6945.41 | Coût : 6455.42 | Durée : 25.82 h
  P007448 | Difficulté : 2649.51 | Coût : 3987.11 | Durée : 15.95 h
  P004862 | Difficulté : 3890.93 | Coût : 4410.74 | Durée : 17.64 h
  P008031 | Difficulté : 6026.85 | Coût : 5489.47 | Durée : 21.96 h
  P007518 | Difficulté : 1209.41 | Coût : 2459.07 | Durée : 9.84 h
  P005117 | Difficulté : 5496.85 | Coût : 6631.35 | Durée : 26.53 h
  P000873 | Difficulté : 933.36 | Coût : 2732.55 | Durée : 10.93 h
  P007790 | Difficulté : 4822.61 | Coût : 5270.51 | Durée : 29.28 h
  P005100 | Difficulté : 1224.52 | Coût : 3129.88 | Durée : 12.52 h
  P000732 | Difficulté : 3636.32 | Coût : 5393.57 | Durée : 21.57 h
  P008001 | Difficulté : 3002.36 | Coût : 4900.91 | Durée : 19.60 h
  P000719 | Difficulté : 3576.58 | Coût : 5349.08 | Durée : 21.40 h
  P005531 | Difficulté : 7875.07 | Coût : 7424.65 | Durée : 29.70 h
  P007639 | Difficulté : 6044.67 | Coût : 5519.52 | Durée : 30.66 h
  P005100 | Di

In [51]:
#On vérifie qu'on respecte le budget et sa répartion
def verify_phase_costs(phases):
    
    total_cost = sum(i.cost for phase in phases.values() for i in phase)
    phase_costs = {}

    print("\nVérification des coûts par phase :")
    for idx, infra_list in phases.items():
        phase_sum = sum(i.cost for i in infra_list)
        phase_costs[idx] = phase_sum
        pct = (phase_sum / total_cost) * 100 if total_cost > 0 else 0
        print(f"Phase {idx} : {phase_sum:.2f} € | {pct:.1f} % du total")

    print("Coût total toutes phases : {total_cost:.2f} €")
    return phase_costs


In [52]:
verify_phase_costs(phases_result)


Vérification des coûts par phase :
Phase 1 : 1248237.91 € | 38.9 % du total
Phase 2 : 592480.94 € | 18.5 % du total
Phase 3 : 602219.30 € | 18.8 % du total
Phase 4 : 765097.16 € | 23.8 % du total
Coût total toutes phases : 3208035.31 €


{1: 1248237.9123755174,
 2: 592480.9411705484,
 3: 602219.3031754037,
 4: 765097.1551690063}

In [70]:
#On génere un fichier csv pour visualiser les différentes phases de reconstruction sur qgis
def export_phases_to_csv_from_data(data, phases, output_file="infra_phases.csv"):

    rows = []

    # Ajouter l'hôpital en phase 0
    df_hopital = data[data["type_batiment"].str.lower() == "hôpital"]
    df_hopital = df_hopital[df_hopital["infra_type"].str.lower() == "a_remplacer"]

    for _, row in df_hopital.iterrows():
        rows.append({
            "infra_id": row["infra_id"],
            "phase_reconstruction": 0,
            "duree_totale": row["duree_totale"],
            "cout_total": row["cout_total"]
        })

    # Ajouter les autres phases
    for phase_idx, infra_list in phases.items():
        for infra in infra_list:
            rows.append({
                "infra_id": infra.infra_id,
                "phase_reconstruction": phase_idx,
                "duree_totale": infra.time,
                "cout_total": infra.cost
            })
    
    df_export = pd.DataFrame(rows)
    df_export.to_csv(output_file, index=False)
    print(f"CSV exporté : {output_file}")


In [73]:
export_phases_to_csv_from_data(data,phases_result)

CSV exporté : infra_phases.csv
